In [ ]:
'''
Before running this code, make sure you have installed the following packages
torch, torchvision, pillow, pandas

Vist this link to download the dataset.
"https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/"

unzip the dataset and place it in the same directory as this code with the following folder structure
dataset
  - test
      - NORMAL
      - PNEUMONIA
  - train
      - NORMAL
      - PNEUMONIA
  - val
      - NORMAL
      - PNEUMONIA
'''

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

classes = {"NORMAL": 0, "PNEUMONIA": 1}

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir.rstrip('/')
        self.transform = transform
        self.annotations = pd.DataFrame([[f"{root_dir}/{fold}/{img_file}", classes[fold]] for fold in os.listdir(self.root_dir) 
                                         for img_file in os.listdir(f"{root_dir}/{fold}")], columns=('image_file', 'label'))

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.annotations.iloc[idx, 1]
        return image, label

# transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# datasets
train_data = CustomDataset(root_dir='dataset/train', transform=train_transform)
test_data = CustomDataset(root_dir='dataset/test', transform=test_transform)

# dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


In [ ]:
# model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Instantiate the model
model = CNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 1):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        running_val_loss = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().view(-1, 1))
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels.float().view(-1, 1)).sum().item()
            running_val_loss += loss.item() * inputs.size(0)
        val_loss = running_val_loss / len(test_data)
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}, Accuracy: {100*correct/total:.2f}%')

    # Save the model after every 5 epochs
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'models/cnn_epoch_{epoch}.pth')
        

In [36]:
# Model evaluation

model = CNN()
weights = torch.load("models/cnn_epoch_45.pth")
model.load_state_dict(weights)
model.to("cuda")
model.eval()

correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to('cuda'), labels.float().reshape(-1, 1).to('cuda')
        outputs = model(inputs)
        preds = torch.round(outputs)
        correct += (preds==labels).sum().item()
        total += len(labels)

print(f"Accuracy: {100*correct/total:.2f}%")

Accuracy: 91.35%
